In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import platform
from datetime import datetime,timedelta


# 운영체제별 한글 폰트 설정
if platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

# dataset

In [2]:
df_1 = pd.read_csv('data/국립자연휴양림 예약 정보(200101~200824).csv')
df_2 = pd.read_csv('data/국립자연휴양림 예약 정보(201014~210322).csv')
df_3 = pd.read_csv('data/국립자연휴양림 예약 정보(210329~210923).csv')
df_4 = pd.read_csv('data/국립자연휴양림 예약 정보(210924~220314).csv')

df_info = pd.read_csv('data/산림청 국립자연휴양림관리소_국립자연휴양림 시설관련 정보_20220906.csv')
df_stat = pd.read_csv('data/산림청 국립자연휴양림관리소_국립자연휴양림 이용객 정산관련 데이터_20200630.csv')
df_describe = pd.read_csv('data/산림청 국립자연휴양림관리소_국립자연휴양림 홍보_20220906.csv')

# pre-processing

In [3]:
df_1 = df_1[['기관ID','기관이름','숙박일자']]
df_1 = df_1.rename(columns={'기관ID':'id','기관이름':'name','숙박일자':'date'})
df_1['id'] = df_1['id'].str.replace("'","")
df_1['name'] = df_1['name'].str.replace("'","")
df_1['date'] = df_1['date'].str.replace("'","")
df_1 = df_1[df_1['date'] != '2(24.7㎡×2실)']  
df_1['date'] = df_1['date'].astype('datetime64')

df_1 = df_1[['date','id','name']]

df_2 = df_2[['기관ID','기관이름','숙박일자']]
df_2 = df_2.rename(columns={'기관ID':'id','기관이름':'name','숙박일자':'date'})
df_2['date'] = df_2['date'].astype('str').astype('datetime64')

df_2 = df_2[['date','id','name']]

df_3 = df_3[['INSTT_ID','INSTT_NM','STNG_DT']]
df_3 = df_3.rename(columns={'INSTT_ID':'id','INSTT_NM':'name','STNG_DT':'date'})
df_3['date'] = df_3['date'].astype('str').astype('datetime64')

df_3 = df_3[['date','id','name']]

df_4 = df_4[['기관아이디','기관이름','숙박일자']]
df_4 = df_4.rename(columns={'기관아이디':'id','기관이름':'name','숙박일자':'date'})
df_4['date'] = df_4['date'].astype('str').astype('datetime64')

df_4 = df_4[['date','id','name']]

# 총 예약 데이터
df = pd.concat([df_1,df_2,df_3,df_4,])
# 2021년 데이터만
df = df[('2021-01-01'<=df.date) & (df.date<='2021-12-31')]

In [4]:
df.isnull().sum()

date    0
id      0
name    0
dtype: int64

In [5]:
df.id = df.id.astype('str').apply(lambda x : x[-3:]).astype('int64')

In [6]:
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.weekday

In [24]:
#df.to_csv('국립휴양림예약데이터.csv')

In [7]:
# 휴양림 별 예약건수
df_id = df.groupby(['id'],as_index=False).agg(resv_count = ('id','count'))

In [8]:
df_info = df_info.drop_duplicates(['아이디'])
df_info = df_info[['아이디','기관명','주소','상품위치위도','상품위치경도']]
df_info = df_info.rename(columns={'아이디':'id','기관명':'name','주소':'address','상품위치위도':'lat','상품위치경도':'lon',})

In [9]:
df_id = pd.merge(df_id,df_info,on='id')

In [10]:
df_stat = df_stat[['휴양림명','총 가동률','주중 가동률','주말 가동률','합계 금액']]
df_stat = df_stat.rename(columns={'휴양림명':'name','총 가동률':'rate','주중 가동률':'wd_rate','주말 가동률':'we_rate','합계 금액':'revenue'})

In [11]:
df_id = pd.merge(df_id,df_stat,on='name')

In [12]:
df_describe = df_describe[['기관명','최대수용인원','기관설명']]
df_describe = df_describe.rename(columns={'기관명':'name','최대수용인원':'capacity','기관설명':'describe'})

In [13]:
df_id = pd.merge(df_id,df_describe,on='name')

In [14]:
df_id = df_id[['id', 'name', 'address', 'lon', 'lat', 'capacity', 'rate', 'wd_rate', 'we_rate', 'resv_count', 'revenue', 'describe']]
df_id[['lon','lat']] = df_id[['lon','lat']].astype('float64')

In [25]:
#df_id.to_csv('국립휴양림정보.csv')

# processing and visualising

In [16]:
import folium
 
df_id_map = folium.Map(location=[36, 127], zoom_start=7)
 
for i in range(len(df_id)):
    folium.Marker(list(df_id.iloc[i][['lon', 'lat']]),
                 popup=df_id.iloc[i][['name']],
                 icon=folium.Icon(color='green')).add_to(df_id_map)

df_id_map


In [17]:
import plotly.graph_objects as go

ratio_total   = list(df_id.loc[:,'resv_count'])
labels_total  = list(df_id.loc[:,'name'])

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels_total, values=ratio_total)])
fig.update_layout(margin=dict(t=20, b=20, l=10, r=20))
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [18]:
df_date = df.groupby(['date','name'],as_index=False).agg(date_resv_count = ('name','count'))
names = list(df_date.name.unique())

In [19]:
fig = px.bar(df_date, x="date", y="date_resv_count" )
fig.show()

In [20]:
for i in names:
    fig = px.bar(df_date[df_date['name'] == i], x="date", y="date_resv_count",title=i) 
    fig.show()


* 일년 내내 이용객 수가 비슷한 남해편백 달음산 변산 속리산 아세안 운악산(주말) 신시도 휴양림이 핫플이라 할수 있겠다.

In [21]:
# 이용률 탑10 
df_id.sort_values(by='rate',ascending=False).head(10)

,id,name,address,lon,lat,capacity,rate,wd_rate,we_rate,resv_count,revenue,describe
32,202,달음산 자연휴양림,(46046) 부산 기장군 일광면 화용길 299-106,35.301665,129.185621,150,89.941,88.000,93.227,5355,35433100,부산광역시 최초의 국립자연휴양림 달음산자연휴양림
8,109,방태산 자연휴양림,(24656) 강원 인제군 기린면 방태산길 241,37.922646,128.398925,600,88.415,85.027,92.908,3312,44748600,수려한 자연경관과 다양한 야생동물을 감상할 수 있는 방태산 자연휴양림
3,104,아세안 자연휴양림,(11518) 경기 양주시 백석읍 기산로 472,37.772979,126.942612,200,83.821,82.296,86.711,6201,94209900,아세안 10개국의 문화 체험을 즐길 수 있는 이색 휴양림 아세안 자연휴양림
22,189,변산 자연휴양림,(56340) 전북 부안군 변산면 변산로 3768,35.581932,126.526669,500,83.821,82.919,85.448,9720,198859700,천혜의 자연경관을 자랑하는 바다가 아름다운 숲 변산 자연휴양림
25,192,남해편백 자연휴양림,(52448) 경남 남해군 삼동면 금암로 658,34.749842,128.017159,1000,82.128,81.362,83.493,11582,160783900,치유의 숲! 편백 향이 가득한 남해편백 자연휴양림
0,101,유명산 자연휴양림,(12473) 경기 가평군 설악면 유명산길 79-53,37.590103,127.482142,3000,79.258,76.503,84.123,21959,187641300,자생식물원을 품은 수도권의 숲속 쉼터 유명산 자연휴양림
10,111,대관령 자연휴양림,(25443) 강원 강릉시 성산면 삼포암길 133,37.714946,128.784168,1000,77.398,75.509,80.787,10408,176473600,"우리나라 최초의 자연휴양림, 대관령 자연휴양림"
1,102,용대 자연휴양림,(24605) 강원 인제군 북면 연화동길 7,38.237016,128.343453,800,75.931,71.742,83.579,6919,57035200,"오염되지 않은 자연 속 휴식 공간, 용대 자연휴양림"
15,141,덕유산 자연휴양림,(55563) 전북 무주군 무풍면 구천동로 530-62,35.907346,127.815650,730,72.708,69.011,79.270,8295,109413100,"주변 경관이 수려한 휴식의 최적지, 덕유산 자연휴양림"
2,103,산음 자연휴양림,(12527) 경기 양평군 단월면 고북길 347,37.597699,127.568919,2000,71.754,65.420,83.439,14275,115218600,자연 그대로의 모습이 보존된 산음 자연휴양림


In [22]:
# 월별 이용객 추이
fig = px.bar(df.groupby(['month'],as_index=False).agg(count = ('id','count')), x="month", y="count" , text_auto=True)
fig.show()

In [23]:
# 요일별 이용객 추이
fig = px.bar(df.groupby(['weekday'],as_index=False).agg(count = ('id','count')), x="weekday", y="count" , text_auto=True)
fig.show()